In [42]:
import torch
import numpy as np

In [43]:
def likelihood_new(z,a,b,sigma,p):
    #z: percept
    #bi-modal, mixture of Gaussian function
    #2 peaks at a and b
    #sigma: equal variance of both modes
    #return: a vectos, p(x|z_i) as a functino of z_i for every sequences running in parallel
    likelihood_ele = torch.exp(-0.5/sigma**2*(torch.reshape(z,(p,-1))-a)**2) + torch.exp(-0.5/sigma**2*(torch.reshape(z,(p,-1))-b)**2)
    return torch.prod(likelihood_ele,axis=1)

In [44]:
def MRF_prior_new(z,lamda,p):
    #return the Gaussian MRF of the perception for each parallel sequence
    #reflection on boundary first
    #lamda: smoothness penalty term
    
    dim = z.shape[1]
    z_extended = torch.cat((torch.reshape(z[:,:,0],(p,-1,1)),z,torch.reshape(z[:,:,-1],(p,-1,1))),axis=2)
    z_extended = torch.cat((torch.reshape(z_extended[:,0,:],(p,1,-1)),z_extended,torch.reshape(z_extended[:,-1,:],(p,1,-1))),axis=1)
    sum_dis = torch.sum(torch.sum((z-z_extended[:,:dim,1:dim+1])**2,axis=1),axis=1)+torch.sum(torch.sum((z-z_extended[:,2:,1:dim+1])**2,axis=1),axis=1)
    sum_dis = sum_dis+torch.sum(torch.sum((z-z_extended[:,1:dim+1,:dim])**2,axis=1),axis=1)+torch.sum(torch.sum((z-z_extended[:,1:dim+1:,2:])**2,axis=1),axis=1)
    return torch.exp(-lamda*sum_dis)

In [50]:
def proposal_function_new(z, p, sigma1 = 1.5):
    #generate a new proposal from the current sample
    #change only one element at a time!
    #sigma1: std of step size in proposal
    dim = z.shape[1]
    z_star = z.clone()
    index = torch.randint(0,dim,(p,2))
    for i in range(p):
        z_star[i,index[i,0],index[i,1]] = z_star[i,index[i,0],index[i,1]] + sigma1*torch.randn(1)
    return z_star

In [46]:
def sample_rejection_new(z,z_star,p,a = 1, b = -1, sigma = 0.3, lamda = 0.05):
    #input: z is the current sample
    #z_star is the proposed sample
    #if sample accepted: return True
    #if rejected: return False
    p_acc = likelihood_new(z_star,a,b,sigma,p)*MRF_prior_new(z_star,lamda,p)/(likelihood_new(z,a,b,sigma,p)*MRF_prior_new(z,lamda,p))
    p_ran = torch.rand(p)
    acc = torch.sign(p_acc-p_ran)
    return acc

In [52]:
def simulation(p,no_samples,device,a = 1, b = -1, sigma = 0.3, lamda = 0.05,dim = 4):
    state_list_par = torch.zeros(no_samples,p,dim,dim)
    state_list_par.to(device)
    #define initial z
    z = (torch.rand(p,dim,dim)-0.5)*2
    z.to(device)
    for i in range(int(no_samples)):
        z_star = proposal_function_new(z,p)
        acc = sample_rejection_new(z,z_star,p,a = 1, b = -1, sigma = 0.3, lamda = 0.05)
        for j in range(p):
            if acc[j] == 1:
                z[j,:,:] = z_star[j,:,:]
        z_a = z.clone()
        state_list_par[i] = z_a
    return state_list_par

In [56]:
p,no_samples = 2,5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
simulation(p,no_samples,device,a = 1, b = -1, sigma = 0.3, lamda = 0.05,dim = 4)[:,0,:,:]

tensor([[[ 0.2346, -0.5301, -0.3254, -0.6535],
         [-0.5009, -0.4365,  0.5328,  0.4943],
         [-0.5273, -0.1767, -0.4649, -0.9543],
         [-0.9549,  0.4064, -0.2428, -0.2414]],

        [[ 0.2346, -0.5301, -0.3254, -0.6535],
         [-0.5009, -0.4365,  0.5328,  0.4943],
         [-0.5273, -0.1767, -0.4649, -0.9543],
         [-0.9549,  0.4064, -0.2428, -0.2414]],

        [[ 0.2346, -0.5301, -0.3254, -0.6535],
         [-0.5009, -0.4365,  0.5328,  0.4943],
         [-0.5273, -0.1767, -0.4649, -0.9543],
         [-0.9549,  0.4064, -0.2428, -0.2414]],

        [[ 0.2346, -0.5301,  1.2918, -0.6535],
         [-0.5009, -0.4365,  0.5328,  0.4943],
         [-0.5273, -0.1767, -0.4649, -0.9543],
         [-0.9549,  0.4064, -0.2428, -0.2414]],

        [[ 0.2346, -0.5301,  1.2918, -0.6535],
         [-0.5009, -0.4365,  0.5328,  0.4943],
         [-0.5273, -0.1767, -0.4649, -0.9543],
         [-0.9549,  0.4064, -0.2428, -0.2414]]])

In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [41]:
z=torch.zeros(2,2,2,2)
z[0]

tensor([[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]])

In [3]:
'''this is the main script to run simulation on Gershman 09's model'''

'''import functions'''
import torch
from sampling_gershman_09 import simulation
from post_proccess import generate_hist

'''the followings are parameters for simulation'''

'''p is the number of parallel sequences, no_samples are the number of samples in each sequence'''
p = 200
no_samples=2200

'''other parameters'''
'''no_bins: number of bins in histogram'''
'''cut_domi: the number of (longest) dominance period to be cutted'''
'''dim: dimension of simulation, set to 4'''
'''definition of a,b,sigma,lamda is in prob_gershman_09'''

no_bins = 1000
cut_domi = 20

'''set device variable here'''
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

s = simulation(p,no_samples,device,a = 1, b = -1, sigma = 0.3, lamda = 0.05,dim = 4)
s.to('cpu')
s.numpy()
hist,bins = generate_hist(s,p,no_samples,no_bins,cut_domi,dim=4,a=1,b=-1)

'''output hist and bins variable to a csv file'''
import numpy.random as random
import numpy as np
rand_ind = random.randint(0,10000)
file_name_hist = 'hist'+str(rand_ind)
file_name_bins = 'bins'+str(rand_ind)
np.savetxt(file_name_hist, hist, delimiter=",")
np.savetxt(file_name_bins, bins, delimiter=",")